# Concatenation, Analysis

In [1]:
import sys
sys.path.insert(0, '../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix, random_unitary
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm
from math import ceil

from loss_functions import *
from optimization import *
from quantum_channel import *
from kraus_channels import *
from quantum_tools import *
from experimental import *
from spam import *
from scipy.stats import gaussian_kde
from quantum_circuits import *

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
def evaluate_channel(channel, inputs, targets):
    d = channel.d
    kl_div = KLDiv()
    loss = kl_div(channel, inputs, targets)
    return np.real(loss)/d

## KL Divergence with Respect to Real Data

In [3]:
model_list = loader(data_path("belem_concatenate_2layer_fitted.model"))
np.random.seed(42)
random.seed(42)

n=3
d = 2**n

R1_fid_list = []
R2_fid_list = []
R3_fid_list = []

U1_fid_list = []
U2_fid_list = []
U3_fid_list = []

T1_fid_list = []
T2_fid_list = []
T12_fid_list = []
T3_fid_list = []

for i in range(10):
    inputs1, targets1, _, _ = loader(data_path(f"belem_concatenate_2layer{i}0"))
    inputs2, targets2, _, _ = loader(data_path(f"belem_concatenate_2layer{i}1"))
    inputs_full, targets_full, _, _ = loader(data_path(f"belem_concatenate_2layer{i}2"))
    
    model1 = model_list[3*i]
    model2 = model_list[3*i+1]
    model_full = model_list[3*i+2]
    
    channel1 = model1.channel
    channel2 = model2.channel
    channel_full = model_full.channel

    channel_concat = channel_to_choi_map([channel1, channel2])
    channel_concat.spam = channel_full.spam
    
    identity_channel1 = IdentityChannel(d)
    identity_channel1.spam = channel1.spam

    identity_channel2 = IdentityChannel(d)
    identity_channel2.spam = channel2.spam

    identity_channel_full = IdentityChannel(d)
    identity_channel_full.spam = channel_full.spam
    
    circuit1 = pqc_basic(n, 2)#.reverse_bits()
    U1 = circuit_to_matrix(circuit1)

    circuit2 = pqc_basic(n, 2)#.reverse_bits()
    U2 = circuit_to_matrix(circuit2)

    circuit_full = deepcopy(circuit1)
    circuit_full = circuit_full.compose(circuit2)
    U_full = circuit_to_matrix(circuit_full)

    U_channel1 = ChoiMapStatic(U1, mode="unitary")
    U_channel1.spam = channel1.spam

    U_channel2 = ChoiMapStatic(U2, mode="unitary")
    U_channel2.spam = channel2.spam

    U_channel_full = ChoiMapStatic(U_full, mode="unitary")
    U_channel_full.spam = channel_full.spam
    
    R1_fid_list.append(evaluate_channel(identity_channel1, inputs1, targets1))
    R2_fid_list.append(evaluate_channel(identity_channel2, inputs2, targets2))
    R3_fid_list.append(evaluate_channel(identity_channel_full, inputs_full, targets_full)) 
    
    U1_fid_list.append(evaluate_channel(U_channel1, inputs1, targets1))
    U2_fid_list.append(evaluate_channel(U_channel2, inputs2, targets2))
    U3_fid_list.append(evaluate_channel(U_channel_full, inputs_full, targets_full))

    T1_fid_list.append(evaluate_channel(channel1, inputs1, targets1))
    T2_fid_list.append(evaluate_channel(channel2, inputs2, targets2))
    T12_fid_list.append(evaluate_channel(channel_concat, inputs_full, targets_full))
    T3_fid_list.append(evaluate_channel(channel_full, inputs_full, targets_full))
    

R1_fid_mean = np.mean(R1_fid_list)
R2_fid_mean = np.mean(R2_fid_list)
R3_fid_mean = np.mean(R3_fid_list)   
    
U1_fid_mean = np.mean(U1_fid_list)
U2_fid_mean = np.mean(U2_fid_list)
U3_fid_mean = np.mean(U3_fid_list)

T1_fid_mean = np.mean(T1_fid_list)
T2_fid_mean = np.mean(T2_fid_list)
T12_fid_mean = np.mean(T12_fid_list)
T3_fid_mean = np.mean(T3_fid_list)

R1_fid_std = np.std(R1_fid_list)
R2_fid_std = np.std(R2_fid_list)
R3_fid_std = np.std(R3_fid_list)   

U1_fid_std = np.std(U1_fid_list)
U2_fid_std = np.std(U2_fid_list)
U3_fid_std = np.std(U3_fid_list)

T1_fid_std = np.std(T1_fid_list)
T2_fid_std = np.std(T2_fid_list)
T12_fid_std = np.std(T12_fid_list)
T3_fid_std = np.std(T3_fid_list)

In [4]:
print(f"R1 KL-Div: {R1_fid_mean:.5f} +-{R1_fid_std:.5f}")
print(f"R2 KL-Div: {R2_fid_mean:.5f} +-{R2_fid_std:.5f}")
print(f"R_full KL-Div: {R3_fid_mean:.4f} +-{R3_fid_std:.5f}")
print("----")
print(f"U1 KL-Div: {U1_fid_mean:.5f} +-{U1_fid_std:.5f}")
print(f"U2 KL-Div: {U2_fid_mean:.5f} +-{U2_fid_std:.5f}")
print(f"U_full KL-Div: {U3_fid_mean:.4f} +-{U3_fid_std:.5f}")
print("----")
print(f"T1 KL-Div: {T1_fid_mean:.5f} +-{T1_fid_std:.5f}")
print(f"T2 KL-Div: {T2_fid_mean:.5f} +-{T2_fid_std:.5f}")
print(f"T12 KL-Div: {T12_fid_mean:.5f} +-{T12_fid_std:.5f}")
print(f"T3 KL-Div: {T3_fid_mean:.5f} +-{T3_fid_std:.5f}")

R1 KL-Div: 0.13294 +-0.01040
R2 KL-Div: 0.12627 +-0.01352
R_full KL-Div: 0.1247 +-0.00988
----
U1 KL-Div: 0.06576 +-0.01772
U2 KL-Div: 0.06506 +-0.01877
U_full KL-Div: 0.0619 +-0.01502
----
T1 KL-Div: 0.00085 +-0.00015
T2 KL-Div: 0.00130 +-0.00095
T12 KL-Div: 0.00533 +-0.00061
T3 KL-Div: 0.00067 +-0.00013


In [11]:
model_list = loader(data_path("belem_concatenate_4layer_fitted.model"))
np.random.seed(42)
random.seed(42)

n=3
d = 2**n

R1_fid_list = []
R2_fid_list = []
R3_fid_list = []

U1_fid_list = []
U2_fid_list = []
U3_fid_list = []

T1_fid_list = []
T2_fid_list = []
T12_fid_list = []
T3_fid_list = []

for i in range(10):
    inputs1, targets1, _, _ = loader(data_path(f"belem_concatenate_4layer{i}0"))
    inputs2, targets2, _, _ = loader(data_path(f"belem_concatenate_4layer{i}1"))
    inputs_full, targets_full, _, _ = loader(data_path(f"belem_concatenate_4layer{i}2"))
    
    model1 = model_list[3*i]
    model2 = model_list[3*i+1]
    model_full = model_list[3*i+2]
    
    channel1 = model1.channel
    channel2 = model2.channel
    channel_full = model_full.channel

    channel_concat = channel_to_choi_map([channel1, channel2])
    channel_concat.spam = channel_full.spam
    
    identity_channel1 = IdentityChannel(d)
    identity_channel1.spam = channel1.spam

    identity_channel2 = IdentityChannel(d)
    identity_channel2.spam = channel2.spam

    identity_channel_full = IdentityChannel(d)
    identity_channel_full.spam = channel_full.spam
    
    circuit1 = pqc_basic(n, 4)
    U1 = circuit_to_matrix(circuit1)

    circuit2 = pqc_basic(n, 4)
    U2 = circuit_to_matrix(circuit2)

    circuit_full = deepcopy(circuit1)
    circuit_full = circuit_full.compose(circuit2)
    U_full = circuit_to_matrix(circuit_full)

    U_channel1 = ChoiMapStatic(U1, mode="unitary")
    U_channel1.spam = channel1.spam

    U_channel2 = ChoiMapStatic(U2, mode="unitary")
    U_channel2.spam = channel2.spam

    U_channel_full = ChoiMapStatic(U_full, mode="unitary")
    U_channel_full.spam = channel_full.spam
    
    R1_fid_list.append(evaluate_channel(identity_channel1, inputs1, targets1))
    R2_fid_list.append(evaluate_channel(identity_channel2, inputs2, targets2))
    R3_fid_list.append(evaluate_channel(identity_channel_full, inputs_full, targets_full)) 
    
    U1_fid_list.append(evaluate_channel(U_channel1, inputs1, targets1))
    U2_fid_list.append(evaluate_channel(U_channel2, inputs2, targets2))
    U3_fid_list.append(evaluate_channel(U_channel_full, inputs_full, targets_full))

    T1_fid_list.append(evaluate_channel(channel1, inputs1, targets1))
    T2_fid_list.append(evaluate_channel(channel2, inputs2, targets2))
    T12_fid_list.append(evaluate_channel(channel_concat, inputs_full, targets_full))
    T3_fid_list.append(evaluate_channel(channel_full, inputs_full, targets_full))
    

R1_fid_mean = np.mean(R1_fid_list)
R2_fid_mean = np.mean(R2_fid_list)
R3_fid_mean = np.mean(R3_fid_list)   
    
U1_fid_mean = np.mean(U1_fid_list)
U2_fid_mean = np.mean(U2_fid_list)
U3_fid_mean = np.mean(U3_fid_list)

T1_fid_mean = np.mean(T1_fid_list)
T2_fid_mean = np.mean(T2_fid_list)
T12_fid_mean = np.mean(T12_fid_list)
T3_fid_mean = np.mean(T3_fid_list)

R1_fid_std = np.std(R1_fid_list)
R2_fid_std = np.std(R2_fid_list)
R3_fid_std = np.std(R3_fid_list)   

U1_fid_std = np.std(U1_fid_list)
U2_fid_std = np.std(U2_fid_list)
U3_fid_std = np.std(U3_fid_list)

T1_fid_std = np.std(T1_fid_list)
T2_fid_std = np.std(T2_fid_list)
T12_fid_std = np.std(T12_fid_list)
T3_fid_std = np.std(T3_fid_list)

In [12]:
print(f"Refrence 1 KL-Div: {evaluate_channel(identity_channel1, inputs1, targets1):.4f}")
print(f"Refrence 2 KL-Div: {evaluate_channel(identity_channel2, inputs2, targets2):.4f}")
print(f"Refrence Full KL-Div: {evaluate_channel(identity_channel_full, inputs_full, targets_full):.4f}")
print("----")
print(f"U1 KL-Div: {evaluate_channel(U_channel1, inputs1, targets1):.4f}")
print(f"U2 KL-Div: {evaluate_channel(U_channel2, inputs2, targets2):.4f}")
print(f"U_full KL-Div: {evaluate_channel(U_channel_full, inputs_full, targets_full):.4f}")
print("----")
print(f"T1 KL-Div: {evaluate_channel(channel1, inputs1, targets1):.4f}")
print(f"T2 KL-Div: {evaluate_channel(channel2, inputs2, targets2):.4f}")
print(f"T2*T1 KL-Div: {evaluate_channel(channel_concat, inputs_full, targets_full):.4f}")
print(f"T_full KL-Div: {evaluate_channel(channel_full, inputs_full, targets_full):.4f}")

Refrence 1 KL-Div: 0.1156
Refrence 2 KL-Div: 0.1321
Refrence Full KL-Div: 0.1100
----
U1 KL-Div: 0.0609
U2 KL-Div: 0.0694
U_full KL-Div: 0.0631
----
T1 KL-Div: 0.0006
T2 KL-Div: 0.0007
T2*T1 KL-Div: 0.0046
T_full KL-Div: 0.0002


In [13]:
model_list = loader(data_path("belem_concatenate_8layer_fitted.model"))
np.random.seed(42)
random.seed(42)

n=3
d = 2**n

R1_fid_list = []
R2_fid_list = []
R3_fid_list = []

U1_fid_list = []
U2_fid_list = []
U3_fid_list = []

T1_fid_list = []
T2_fid_list = []
T12_fid_list = []
T3_fid_list = []

for i in range(10):
    inputs1, targets1, _, _ = loader(data_path(f"belem_concatenate_8layer{i}0"))
    inputs2, targets2, _, _ = loader(data_path(f"belem_concatenate_8layer{i}1"))
    inputs_full, targets_full, _, _ = loader(data_path(f"belem_concatenate_8layer{i}2"))
    
    model1 = model_list[3*i]
    model2 = model_list[3*i+1]
    model_full = model_list[3*i+2]
    
    channel1 = model1.channel
    channel2 = model2.channel
    channel_full = model_full.channel

    channel_concat = channel_to_choi_map([channel1, channel2])
    channel_concat.spam = channel_full.spam
    
    identity_channel1 = IdentityChannel(d)
    identity_channel1.spam = channel1.spam

    identity_channel2 = IdentityChannel(d)
    identity_channel2.spam = channel2.spam

    identity_channel_full = IdentityChannel(d)
    identity_channel_full.spam = channel_full.spam
    
    circuit1 = pqc_basic(n, 8)
    U1 = circuit_to_matrix(circuit1)

    circuit2 = pqc_basic(n, 8)
    U2 = circuit_to_matrix(circuit2)

    circuit_full = deepcopy(circuit1)
    circuit_full = circuit_full.compose(circuit2)
    U_full = circuit_to_matrix(circuit_full)

    U_channel1 = ChoiMapStatic(U1, mode="unitary")
    U_channel1.spam = channel1.spam

    U_channel2 = ChoiMapStatic(U2, mode="unitary")
    U_channel2.spam = channel2.spam

    U_channel_full = ChoiMapStatic(U_full, mode="unitary")
    U_channel_full.spam = channel_full.spam
    
    R1_fid_list.append(evaluate_channel(identity_channel1, inputs1, targets1))
    R2_fid_list.append(evaluate_channel(identity_channel2, inputs2, targets2))
    R3_fid_list.append(evaluate_channel(identity_channel_full, inputs_full, targets_full)) 
    
    U1_fid_list.append(evaluate_channel(U_channel1, inputs1, targets1))
    U2_fid_list.append(evaluate_channel(U_channel2, inputs2, targets2))
    U3_fid_list.append(evaluate_channel(U_channel_full, inputs_full, targets_full))

    T1_fid_list.append(evaluate_channel(channel1, inputs1, targets1))
    T2_fid_list.append(evaluate_channel(channel2, inputs2, targets2))
    T12_fid_list.append(evaluate_channel(channel_concat, inputs_full, targets_full))
    T3_fid_list.append(evaluate_channel(channel_full, inputs_full, targets_full))
    

R1_fid_mean = np.mean(R1_fid_list)
R2_fid_mean = np.mean(R2_fid_list)
R3_fid_mean = np.mean(R3_fid_list)   
    
U1_fid_mean = np.mean(U1_fid_list)
U2_fid_mean = np.mean(U2_fid_list)
U3_fid_mean = np.mean(U3_fid_list)

T1_fid_mean = np.mean(T1_fid_list)
T2_fid_mean = np.mean(T2_fid_list)
T12_fid_mean = np.mean(T12_fid_list)
T3_fid_mean = np.mean(T3_fid_list)

R1_fid_std = np.std(R1_fid_list)
R2_fid_std = np.std(R2_fid_list)
R3_fid_std = np.std(R3_fid_list)   

U1_fid_std = np.std(U1_fid_list)
U2_fid_std = np.std(U2_fid_list)
U3_fid_std = np.std(U3_fid_list)

T1_fid_std = np.std(T1_fid_list)
T2_fid_std = np.std(T2_fid_list)
T12_fid_std = np.std(T12_fid_list)
T3_fid_std = np.std(T3_fid_list)

IndexError: list index out of range

In [20]:
print(f"Refrence 1 KL-Div: {evaluate_channel(identity_channel1, inputs1, targets1):.4f}")
print(f"Refrence 2 KL-Div: {evaluate_channel(identity_channel2, inputs2, targets2):.4f}")
print(f"Refrence Full KL-Div: {evaluate_channel(identity_channel_full, inputs_full, targets_full):.4f}")
print("----")
print(f"U1 KL-Div: {evaluate_channel(U_channel1, inputs1, targets1):.4f}")
print(f"U2 KL-Div: {evaluate_channel(U_channel2, inputs2, targets2):.4f}")
print(f"U_full KL-Div: {evaluate_channel(U_channel_full, inputs_full, targets_full):.4f}")
print("----")
print(f"T1 KL-Div: {evaluate_channel(channel1, inputs1, targets1):.4f}")
print(f"T2 KL-Div: {evaluate_channel(channel2, inputs2, targets2):.4f}")
print(f"T2*T1 KL-Div: {evaluate_channel(channel_concat, inputs_full, targets_full):.4f}")
print(f"T_full KL-Div: {evaluate_channel(channel_full, inputs_full, targets_full):.4f}")

Refrence 1 KL-Div: 0.1149
Refrence 2 KL-Div: 0.1149
Refrence Full KL-Div: 0.1205
----
U1 KL-Div: 0.0593
U2 KL-Div: 0.0637
U_full KL-Div: 0.0553
----
T1 KL-Div: 0.0004
T2 KL-Div: 0.0006
T2*T1 KL-Div: 0.0018
T_full KL-Div: 0.0002
